# Ontology Building

## 0. Setup

 The utils.py file includes the relevant imports and methods.

In [ ]:
from utils import *

Define file paths for output and data directories

In [ ]:
data_path = "./data"  # You must define this path.
out_path = "./output"  # Path to the directory for saving output files.

# Make sure you created the following files using coreference resolution algorithm output and the cleaning script:
weighted_pairs_file = "weighted_pairs.csv"  # Graph edges
phrase2clean_file = "phrase2clean.pkl"  # A dict containing the clean phrases of their original occurences

phrase_occurences_path = os.path.join(data_path, weighted_pairs_file)  
phrase2clean_path = os.path.join(data_path, phrase2clean_file)  

betweeness_path = os.path.join(data_path, "betweenness_full_graph_#k=500.json")  # Optional - run betweeeness centrality separately (may take some time)

## 1. Coreference Graph Construction

Load graph data.

In [ ]:
if not os.path.exists(phrase_occurences_path):
    print("missing file!", phrase_occurences_path)
else:
    df = pd.read_csv(phrase_occurences_path)

Create a weighted undirected graph from the edges in 'df'.

In [ ]:
edges = []
for index, row in df.iterrows():
    edges.append((row['Node1'], row['Node2'], {"weight": row["Weight"]}))

G_w = nx.Graph()
G_w.add_edges_from(edges)  
G_w.number_of_nodes(), G_w.number_of_edges()


## 2. Ontology Extration

### 2.1 Betweenness Centrality Approximation

Run betweenees algorithms with approximation (based on k as number of pivots).

In [ ]:
if os.path.exists(betweeness_path):
    with open(betweeness_path, "r") as file:
        betweenness_centrality_res = json.load(file)
else:
    betweenness_centrality_res = nx.betweenness_centrality(G_w, backend="parallel", weight="weight", k=500)
    with open(betweeness_path, "w") as file:
        json.dump(betweenness_centrality_res, file)  

Tag the edges with respect to the betweeness results.

In [ ]:
for u, v in G_w.edges():
    if betweenness_centrality_res[u] == 0 and betweenness_centrality_res[v] == 0:  # Identity tags
        nx.set_edge_attributes(G_w, {(u, v):{"tag": TAGS["I-1"]}})
    else:  # Hierarchy tags
        parent = u if betweenness_centrality_res[u] > betweenness_centrality_res[v] else v
        child = u if parent != u else v
        nx.set_edge_attributes(G_w, {(parent, child):{"tag": TAGS["H-1"], "dir":(parent, child)}})
        

### 2.2 Betweeness Centrality - Completions

#### Classifying Phrase Type

Create a dictionary to store occurrences of phrases categorized by 'name' or 'noun'. A phrase counts as a noun if it has at least one occurrence without a capital letter, otherwise it counts as a name.

In [ ]:
if not os.path.exists(phrase2clean_path):
    print("missing file!", phrase2clean_path)
else:
    with open(phrase2clean_path, "rb") as f:
        phrase2clean = pickle.load(f)
        clean2phrase = {v: [k for k in phrase2clean if phrase2clean[k] == v] for k, v in phrase2clean.items()}
        tagged_phrases =  {phrase: "name" for phrase in clean2phrase.keys()}
        for phrase, occurences in clean2phrase.items():
            for occur in occurences:
                if occur.islower():
                    tagged_phrases[phrase.lower()] = "noun"
    print(Counter(tagged_phrases.values()))

#### Correcting Directionality

Update edge direction in the graph 'G_w' based on the tags of connected nodes.

In [ ]:
for u, v in G_w.edges:
    data = G_w.get_edge_data(u, v) 
    if "dir" not in data: # Filter non-hierarchical tags (identity, noise)
        continue
    if u in tagged_phrases and v in tagged_phrases: 
        tags = {tagged_phrases[u], tagged_phrases[v]}
        curr_dir = data["dir"] 
        parent, child = curr_dir  # Unpack the current direction
        tags_curr_dir = (tagged_phrases[parent], tagged_phrases[child]) 
        if tags_curr_dir == ('name', 'noun'): 
            nx.set_edge_attributes(G_w, {(u, v): {"tag": TAGS["H-2"], "dir": (child, parent)}})


#### Correcting Tags

Make the graph directed and use it to locate name phrases the have children.

In [ ]:
G_dir = get_directed_graph(G_w)
names_having_children = [ph for ph in tagged_phrases if tagged_phrases[ph] == "name" and ph in G_dir and len(list(G_dir.successors(ph))) > 1]


For each split node save the relevant edges to modify and the node for removal.

In [ ]:
child_edges = []
parent_edges = []
nodes_to_remove = set()

k_neighbors = 2

for ph in tqdm(names_having_children[:10]):
    
    children = list(G_dir.successors(ph))
    embeds = get_embeds(children)
    
    knn = NearestNeighbors(n_neighbors=k_neighbors)
    knn.fit(embeds)
    _, indices = knn.kneighbors(embeds)
    
    knn_graph = nx.Graph()
    for i, child in enumerate(children):
        for neighbor_idx in indices[i]:
            knn_graph.add_edge(child, children[neighbor_idx])
    
    partition = community_louvain.best_partition(knn_graph)

    concepts = group_strings_by_class(children, list(partition.values()))

    common_parents = set(G_dir.predecessors(ph))
    
    if len(concepts) == 1:  # Case 1: single community detected
        for child_group in concepts.values():
            for child in child_group:
                child_edges.append((ph, child))
                
                if child in G_dir.nodes:
                    common_parents.update(G_dir.predecessors(child))
        
        for parent in common_parents:
            parent_edges.append((parent, ph))
    
    else:  # Case 2: multiple communities detected
        for k, v in concepts.items():
            for ind, child in enumerate(v):
                curr_ph = f"{ph} [[[ {k + ind + 1} ]]]"
                child_edges.append((curr_ph, child))
                
                if child in G_dir.nodes:
                    common_parents.update(G_dir.predecessors(child))
            
            for parent in common_parents:
                parent_edges.append((parent, curr_ph))
            
            common_parents = set(G_dir.predecessors(ph))

        nodes_to_remove.add(ph)


In [ ]:
for u, v in child_edges:
    G_w.add_edge(u, v, weight=1.5, tag=TAGS["I-2"])

for u, v in parent_edges:
    G_w.add_edge(u, v, weight=1.5, tag=TAGS["H-1"])

G_w.remove_nodes_from(nodes_to_remove)

## 3. Cleaning Noisy Edges

Calculate the weighted degree for each node in the graph 'G_w', and the total sum of all weighted degrees.

In [ ]:
weighted_degrees = {node: sum(weight for _, _, weight in G_w.edges(node, data='weight')) for node in G_w.nodes}
all_occur = sum(weighted_degrees.values())


Calculate the PMI for each edge (u, v) in the graph based on weighted degrees and edge weights.

In [ ]:
pmi = defaultdict()
for u, v in G_w.edges:
    w = G_w.get_edge_data(u, v)["weight"]
    pmi[(u, v)] = calculate_pmi(weighted_degrees[u], weighted_degrees[v], w, all_occur)


Assign a noise tag to edges in the graph 'G_w' if their PMI value is less than or equal to zero.

In [ ]:
for (u, v), val in pmi.items():
    if val <= 0: 
        nx.set_edge_attributes(G_w, {(u, v): {"tag": TAGS["N"]}}) 


## 4. The Resulted Graph

In [ ]:
get_tags_status(G_w)

Generate a DataFrame from the graph 'G_w' containing the edges, weights, and tags.

In [ ]:
df = get_df_of_graph(G_w)
# df.to_csv(os.path.join(out_path, 'full_graph_tagged.csv'))

In [ ]:
if not os.path.exists(out_path): 
    os.makedirs(out_path) 

Update concepts in the graph 'G_w' and obtain updated mappings.

In [ ]:
dict_concepts_updated, phrase2id_updated = update_concepts(G_w)

Generate a directed acyclic graph (DAG) using the df representing the graph 'G_w'.

In [ ]:
dag, _ = get_dag(df, phrase2id_updated)

Invert the DAG to create a mapping of children to their respective parents.

In [ ]:
inverted_dag = invert_dag(dag)

Save the resulting ontology files.

In [ ]:
with open(os.path.join(out_path, "dict_concepts_updated.pkl"), "wb") as f:
    pickle.dump(dict_concepts_updated, f)
with open(os.path.join(out_path, "phrase2id_updated.pkl"), "wb") as f:
    pickle.dump(phrase2id_updated, f)
with open(os.path.join(out_path, "children_updated.pkl"), "wb") as f:
    pickle.dump(dag, f)
with open(os.path.join(out_path, "parents_updated.pkl"), "wb") as f:
    pickle.dump(inverted_dag, f)